In [1]:
import pandas as pd
import re
import glob
import os
import pyodbc
pd.set_option('display.max_rows', None)

# rename column name
#
def cr_col_nm(lst_col):
    new_lst_col=[]    
    for idx,val in enumerate(lst_col):
        if re.search("[^A-Za-z0-9]",val):
            new_col = re.sub('[^A-Za-z0-9]', '', val)
            new_lst_col.append(new_col)

        else:
             new_lst_col.append(val)
                
    return new_lst_col

# get the latest source file (xlsx)
# different file date and different groups
#
def get_latest_xlsx(spec_pth):
    list_of_files = glob.glob(spec_pth+'*.xlsx') 
    latest_file = max(list_of_files, key=os.path.getctime)
    
    return latest_file

def init_char(txt):
    result = txt[0]+'.'
    return result

In [2]:
root_pth = 'D:\\mix\\data_Eng_nots\\DE_interview_challenges\\2021_JUL_PRIVIA-HEALTH_US\\PythonTestQuestions\\'
src_file = get_latest_xlsx(root_pth)

# get date of file and prodiver group name
splt_lst = src_file.split('\\')
file_nm = splt_lst[-1]
file_nm = file_nm.replace('.xlsx','')
# extract date of file
date_file_lst = re.findall(r'\d+', file_nm)
date_file = date_file_lst[0]
# extract prodiver group name
group_lst = re.findall(r'[A-z]+', file_nm)
group_nm = ' '.join(group_lst)

In [3]:
df = pd.read_excel(src_file, header=1)
df = df.dropna(axis=1, how='all') # drop na columns
df = df.dropna(axis=0, how='all') # drop na row

# fill value on na cells
df = df.fillna(method='ffill', axis=1) 
# remove some unnessarry information
df = df[:-2] 

df = df.transpose() # convert x to y
df = df.reset_index()
df = df.drop(['index'], axis=1)
df_demog = df.copy()

In [4]:
# ------------- 1 point A -------------
# select data only Demographics 
df_demog = df_demog[df_demog[0] == 'Demographics']
df_demog = df_demog.transpose() # convert x to y

# convert 1st row to column
df_demog.drop (index = 0, inplace= True)
df_demog.columns = df_demog.iloc[0]

# ------------- 1 point B -------------
# clear and rename col name follwing requirement  
col_df = cr_col_nm(list(df_demog.columns)) 
df_demog = df_demog[1:]
df_demog.columns = col_df

# convert data tpye
df_demog["DOB1"] = pd.to_datetime(df_demog["DOB1"])

# ------------- 1 point E -------------
# convert sex value 
df_demog['Sex'] = df_demog['Sex'].map({1:'F', 0:'M'})

# ------------- 1 point C -------------
# Include fields in the data table (date of the file, the provider group)
df_demog['date_of_file'] = date_file
df_demog['date_of_file'] = pd.to_datetime(df_demog['date_of_file'], format='%m%d%y')
df_demog['provider_group'] = group_nm

# ------------- 1 point D -------------
df_demog['MiddleName'] = df_demog['MiddleName'].map(init_char)

# ------------- additional (spilt array in FavoriteColor) -------------
# check format data with group by count
# df_demog.groupby(['FavoriteColor']).size().reset_index(name='counts') 
df_demog = df_demog.set_index(['ID', 'FirstName', 'MiddleName', 'LastName', 'DOB1', 'Sex', 'date_of_file', 'provider_group']) \
        .apply(lambda x: x.str.split(',') \
        .explode()).reset_index()
df_demog['FavoriteColor'] = df_demog['FavoriteColor'].str.strip()

In [5]:
df_demog.head()

,ID,FirstName,MiddleName,LastName,DOB1,Sex,date_of_file,provider_group,FavoriteColor
0,0000012275,Myrl,M.,Isom,1932-09-04 01:02:23.539,M,2018-11-30,Privia Family Medicine,Unknown
1,0000034157,Emelda,E.,Bain,1919-07-25 18:12:55.931,M,2018-11-30,Privia Family Medicine,Green
2,0000046629,Maryrose,M.,Crowe,1905-07-13 09:32:37.843,F,2018-11-30,Privia Family Medicine,Yellow
3,0000046629,Maryrose,M.,Crowe,1905-07-13 09:32:37.843,F,2018-11-30,Privia Family Medicine,Orange
4,0000047313,Agnus,A.,Wahl,1905-03-08 14:31:37.670,M,2018-11-30,Privia Family Medicine,Green


In [8]:
df_qr = df.copy()
df_qr = df_qr.transpose()
df_qr.columns = df_qr.iloc[1]
df_qr = df_qr[2:]
col_df = cr_col_nm(list(df_qr.columns)) # clear and rename col name follwing requirement
df_qr.columns = col_df
df_qr = df_qr.drop(['FirstName','MiddleName','LastName','DOB1','Sex','FavoriteColor'], axis=1)

In [9]:
# ------------- 2 point A -------------
df_qr_q1 = df_qr.copy()
df_qr_q1 = df_qr_q1.drop(['AttributedQ2','RiskQ2'], axis=1)
df_qr_q1.rename(columns={'AttributedQ1':'AttributedFlag','RiskQ1':'RiskScore'}, inplace=True)
df_qr_q1['Quarter'] = 1

df_qr_q2 = df_qr.copy()
df_qr_q2 = df_qr_q2.drop(['AttributedQ1','RiskQ1'], axis=1)
df_qr_q2.rename(columns={'AttributedQ2':'AttributedFlag','RiskQ2':'RiskScore'}, inplace=True)
df_qr_q2['Quarter'] = 2

In [10]:
# ------------- 2 point B -------------
df_qr_final = pd.concat([df_qr_q2, df_qr_q1], axis=0)
df_qr_final = df_qr_final[df_qr_final['RiskIncreasedFlag'] == 'Yes']
df_qr_final = df_qr_final[['ID','Quarter','AttributedFlag','RiskScore']]
df_qr_final['date_of_file'] = date_file
df_qr_final['date_of_file'] = pd.to_datetime(df_qr_final['date_of_file'], format='%m%d%y')

In [11]:
df_qr_final.head()

,ID,Quarter,AttributedFlag,RiskScore,date_of_file
2,0000012275,2,No,0.462527,2018-11-30
3,0000034157,2,Yes,0.641157,2018-11-30
4,0000046629,2,Yes,0.625855,2018-11-30
6,0000062025,2,Yes,0.845459,2018-11-30
10,0000081702,2,Yes,0.974527,2018-11-30
